In [1]:
import carla
import random

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
# client.set_timeout(500)
world = client.get_world()

In [ ]:
#python config.py --map Town02

In [2]:
spawn_points = world.get_map().get_spawn_points()
spawn_0 = spawn_points[0]
spawn_points = spawn_points[1:]
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
my_vehicle = world.spawn_actor(random.choice(vehicle_blueprints), spawn_0)

In [ ]:
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints, random.choice(spawn_points)))
for vehicle in world.get_actors().filter('vehicle'):
    vehicle.set_autopilot(True)

In [3]:
blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
# Modify the attributes of the blueprint to set image resolution and field of view.
blueprint.set_attribute('image_size_x', '1920')
blueprint.set_attribute('image_size_y', '1080')
blueprint.set_attribute('fov', '110')
#blueprint.set_attribute('sensor_tick', '1.5')
blueprint.set_attribute('blur_amount', '0')
#motion_blur_max_distortion
blueprint.set_attribute('motion_blur_max_distortion', '0')
# Set the time in seconds between sensor captures
blueprint.set_attribute('sensor_tick', '3.0')
blueprint.set_attribute('shutter_speed', '100.0')
transform = carla.Transform(carla.Location(x=0.8, z=1.7))
sensor = world.spawn_actor(blueprint, transform, attach_to=my_vehicle)
#sensor.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))


In [6]:
spectator = world.get_spectator()
spectator.set_transform(spawn_0)

In [4]:
my_vehicle.set_autopilot(True)

## BB Stuff
Realistically we can only get 3 classes TrafficSigns, Pedestrians, and Vehicles

In [2]:
import queue
world  = client.get_world()
bp_lib = world.get_blueprint_library()

# spawn vehicle
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
spawn_points = world.get_map().get_spawn_points()
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
#camera_bp.set_attribute('sensor_tick', '3.0')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 5
world.apply_settings(settings)

# Get the map spawn points
spawn_points = world.get_map().get_spawn_points()

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [3]:
import numpy as np
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [4]:
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [55]:
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))

# Remember the edge pairs
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]

In [17]:
actor_list = world.get_actors()
for i in actor_list:
    print(i.get_location(), i.type_id)

Location(x=-5.076151, y=184.659592, z=3.424569) spectator
Location(x=147.110016, y=234.222107, z=0.220000) traffic.traffic_light
Location(x=129.550003, y=226.559998, z=0.220000) traffic.traffic_light
Location(x=122.480003, y=243.816986, z=0.220000) traffic.traffic_light
Location(x=121.369987, y=194.089951, z=0.220000) traffic.traffic_light
Location(x=138.929977, y=201.819962, z=0.220000) traffic.traffic_light
Location(x=146.000000, y=184.829971, z=0.220000) traffic.traffic_light
Location(x=30.959990, y=194.289963, z=0.220000) traffic.traffic_light
Location(x=48.519978, y=203.279953, z=0.220000) traffic.traffic_light
Location(x=58.050007, y=185.029984, z=0.220000) traffic.traffic_light
Location(x=186.950058, y=176.769974, z=0.220000) traffic.traffic_light
Location(x=179.220016, y=194.329910, z=0.220000) traffic.traffic_light
Location(x=196.210022, y=201.400040, z=0.220000) traffic.traffic_light
Location(x=186.950058, y=226.179977, z=0.220000) traffic.traffic_light
Location(x=179.220016,

In [12]:
location = world.get_level_bbs(carla.CityObjectLabel.TrafficSigns)[0].location
location.x, location.y

(108.19998168945312, 333.25)

## Some issues with conversion from 3D bb to 2

In [71]:
import cv2
# world.tick()
# image = image_queue.get()
img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
actor_list = world.get_actors()
# Get camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
for bb in bounding_box_set:
    dist = bb.location.distance(vehicle.get_transform().location)
    if dist < 30:
        forward_vec = vehicle.get_transform().get_forward_vector()
        ray = bb.location - vehicle.get_transform().location

        if forward_vec.dot(ray) > 1:
            lab = ""
            location_x, location_y = bb.location.x, bb.location.y
            print(location_x, location_y)
            for i in actor_list:
                tmp_x, tmp_y = i.get_location().x, i.get_location().y
                if round(location_x) == round(tmp_x) and round(location_y) == round(tmp_y):
                    lab = i.type_id
                    print(lab)



            p1 = get_image_point(bb.location, K,world_2_camera)
            verts = [v for v in bb.get_world_vertices(carla.Transform())]
            x_max = -10000
            x_min = 10000
            y_max = -10000
            y_min = 10000
            for vert in verts:
                p = get_image_point(vert, K, world_2_camera)
                print(p)
                # Find the rightmost vertex
                if p[0] > x_max:
                    x_max = p[0]
                # Find the leftmost vertex
                if p[0] < x_min:
                    x_min = p[0]
                # Find the highest vertex
                if p[1] > y_max:
                    y_max = p[1]
                # Find the lowest  vertex
                if p[1] < y_min:
                    y_min = p[1]
            cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
cv2.imwrite("out/test.png", img)

-1.0256996154785156 202.78057861328125
traffic.traffic_light
[1036.21339947  542.40892302]
[1036.22054887  501.62342835]
[1037.95971435  542.48659962]
[1037.96718592  500.76686204]
[1045.46818968  542.40900964]
[1045.47615976  501.62354038]
[1047.42647825  542.48668817]
[1047.43480853  500.76697719]
-9.880000114440918 208.70999145507812
traffic.speed_limit.30
[666.21687644 554.77644368]
[666.19617805 521.87910046]
[667.80894394 554.69103204]
[667.78847101 521.97199606]
[633.32007623 554.7761632 ]
[633.29702474 521.87874709]
[635.09044533 554.69075288]
[635.06764473 521.97164481]


True

In [5]:
from time import sleep
sensor.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))
sleep(10)
sensor.stop()

In [5]:
sensor.stop()

In [5]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [6]:
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))